In [1]:
import os 
%pwd


'u:\\mlflow-dvc-cancer\\research'

In [2]:
os.chdir("../")
%pwd

'u:\\mlflow-dvc-cancer'

In [3]:
from dotenv import load_dotenv 
load_dotenv()

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [4]:
# Now we will be integrating mlflow and dagshub in this
import dagshub
dagshub.init(repo_owner='utkarsh-iitbhu', repo_name='mlflow-dvc-classification-tf', mlflow=True)

Accessing as utkarsh-iitbhu

Initialized MLflow to track repo "utkarsh-iitbhu/mlflow-dvc-classification-tf"

Repository utkarsh-iitbhu/mlflow-dvc-classification-tf initialized!

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path # Model path
    training_data: Path # Data Path
    all_params: dict # Get all the params as a dict
    mlflow_uri: str # mlflow uri to target
    params_image_size: list 
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig( # Create a class
            path_of_model="artifacts/training/model.h5", # Give path as they exist
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/utkarsh-iitbhu/mlflow-dvc-classification-tf.mlflow",
            all_params=self.params, # get all the params
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
        # This link has code for ImageDataGenerator 
        
        # List all the params to tune in model
        # These params of dict will go to ImageDataGenerator
        datagenerator_kwargs = dict( # We can add other params as well
            rescale=1./255, # Rescaling the images
            validation_split=0.3 # As we are not having diff test data
        )
        
        # This is for flow_from_directory
        dataflow_kwargs = dict( # This is same for train and validation sets
            target_size= self.config.params_image_size[:-1],#(224,224)
            batch_size= self.config.params_batch_size, # 16
            interpolation="bilinear", # This is for resizing
        )
        
        # Now create your datagenerator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs # Fill in the dict of values here
        )
        
        # Now create the flow_from_directory | how to get data from dir
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data, # Comes from Inheritance
            subset="validation", # This is for validation data
            shuffle=False,
            **dataflow_kwargs # This will load the other params
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self): # Evaluate on my data
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator() # Split the data
        # Evaluates the loaded model using the validation data
        self.score = self.model.evaluate(self.valid_generator) # keras.evaluate
        # Stores the evaluation results in self.score
        self.save_score()

    def save_score(self): # 2 metrics are there in score
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri) # Track uri
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        # Retrieves the current tracking URI and parses it to determine the scheme (e.g., http, https, file).
        
        with mlflow.start_run(): # Start run
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
    

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-11 18:26:56,055: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-11 18:26:56,063: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-11 18:26:56,066: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 91s 13s/step - loss: 2.6545 - accuracy: 0.4412
[2024-07-11 18:28:28,859: INFO: common: json file saved at: scores.json]


2024/07/11 18:28:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-11 18:28:41,530: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\utkar\AppData\Local\Temp\tmpaduslkse\model\data\model\assets
[2024-07-11 18:28:45,105: INFO: builder_impl: Assets written to: C:\Users\utkar\AppData\Local\Temp\tmpaduslkse\model\data\model\assets]


u:\mlflow-dvc-cancer\cancer\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/07/11 18:30:20 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
